# Deep Learning approach to Pneumonia Detection and Classification from Chest X-Ray

### Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense , Conv2D , MaxPooling2D , Dropout , BatchNormalization , Flatten , MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

### Build the CNN

In [ ]:
model = Sequential()

model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (256,256,3)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())

model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                        

In [ ]:
from tensorflow.keras.optimizers import  Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

### Data Preprocessing

In [ ]:
# Paths
train_dir = 'dataset//train'
test_dir = 'dataset//test'
val_dir = 'dataset//val'

In [ ]:
# ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(256,256),
        batch_size=32,
        class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(256,256),
        batch_size=32,
        class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(256,256),
        batch_size=32,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


### Traning

In [ ]:
history = model.fit(train_generator,
                         steps_per_epoch = 163,
                         epochs = 50,
                         validation_data = val_generator,
                         validation_steps = 624)

Epoch 1/50
163/163 [==============================] - 64s 390ms/step - loss: 0.1298 - accuracy: 0.9534 - val_loss: 36.5900 - val_accuracy: 0.5000
Epoch 2/50
163/163 [==============================] - 68s 417ms/step - loss: 0.1337 - accuracy: 0.9519
Epoch 3/50
163/163 [==============================] - 65s 398ms/step - loss: 0.1205 - accuracy: 0.9553
Epoch 4/50
163/163 [==============================] - 64s 394ms/step - loss: 0.1279 - accuracy: 0.9588
Epoch 5/50
163/163 [==============================] - 62s 380ms/step - loss: 0.1069 - accuracy: 0.9590
Epoch 6/50
163/163 [==============================] - 66s 401ms/step - loss: 0.1178 - accuracy: 0.9609
Epoch 7/50
163/163 [==============================] - 64s 392ms/step - loss: 0.1128 - accuracy: 0.9586
Epoch 8/50
163/163 [==============================] - 64s 395ms/step - loss: 0.1035 - accuracy: 0.9640
Epoch 9/50
163/163 [==============================] - 66s 406ms/step - loss: 0.1002 - accuracy: 0.9659
Epoch 10/50
163/163 [=========

In [ ]:
test_accuracy = model.evaluate(test_generator,steps=624)
train_accuracy = model.evaluate(train_generator,steps=163)
print('The testing accuracy is :',test_accuracy[1]*100, '%')
print('The training accuracy is :',train_accuracy[1]*100, '%')

163/163 [==============================] - 65s 397ms/step - loss: 2.2375 - accuracy: 0.7364
The testing accuracy is : 76.76281929016113 %
The training accuracy is : 73.63880276679993 %


In [ ]:
model.save('models//Pneumonia_Model.h5')